3.2 Analyze Research Diversity

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os

In [ ]:
# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Load Excel file
excel_file = '/content/Task-1.xlsx'
researchers = pd.ExcelFile(excel_file).sheet_names

In [ ]:
# Function to compute diversity
def compute_diversity(abstracts):
    if len(abstracts) < 2:
        return 0.0, "N/A"

    # Generate embeddings
    embeddings = model.encode(abstracts, show_progress_bar=False)

    # Compute pairwise cosine similarity
    similarity_matrix = cosine_similarity(embeddings)

    # Get upper triangle of similarity matrix (excluding diagonal)
    similarities = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]

    # Average similarity
    avg_similarity = similarities.mean() if similarities.size > 0 else 0.0

    # Determine diversity
    if avg_similarity > 0.8:
        diversity = "Low Diversity"
    elif avg_similarity > 0.5:
        diversity = "Medium Diversity"
    else:
        diversity = "High Diversity"

    return round(avg_similarity, 2), diversity

In [ ]:
# Process each researcher
diversity_data = []
for researcher in researchers:
    df = pd.read_excel(excel_file, sheet_name=researcher)
    abstracts = df['Abstract'].dropna().astype(str).tolist()

    if abstracts:
        avg_similarity, diversity = compute_diversity(abstracts)
        diversity_data.append([researcher, avg_similarity, diversity])

In [ ]:
# Save to summary sheet
diversity_df = pd.DataFrame(diversity_data, columns=['Researcher', 'Average Similarity', 'Diversity Score'])
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    diversity_df.to_excel(writer, sheet_name='Author_Diversity', index=False)

print("Diversity analysis completed and saved to 'Author_Diversity' sheet.")

Diversity analysis completed and saved to 'Author_Diversity' sheet.


In [ ]:
print(author_diversity_df)

              Researcher  Average Similarity   Diversity Score
0           Songlin Yang                0.49    High Diversity
1     Samuel Cahyawijaya                0.48    High Diversity
2          Xian-Ling Mao                0.41    High Diversity
3   Kaushal Kumar Maurya                0.41    High Diversity
4              Bowei Zou                0.47    High Diversity
5             Sheng Shen                0.50    High Diversity
6         xian-Ling Mao-                0.41    High Diversity
7            Jiatong Shi                0.50  Medium Diversity
8       Malte Ostendorff                0.43    High Diversity
9           Jingang Wang                0.35    High Diversity
10            Indira Sen                0.35    High Diversity
11      Momchil Hardalov                0.40    High Diversity
12    Eleftheria Briakou                0.51  Medium Diversity
13            Yusheng Su                0.42    High Diversity
14            Yilin Yang                0.55  Medium Di